# 토픽모델 LDA 실행하기

**Install Library & font**

konlpy :   
- https://konlpy.org/ko/latest/#  
- https://konlpy-ko.readthedocs.io/ko/v0.4.4/install/  
- 형태소 분석 및 품사 태깅을 위한 Library

pyldavis : 
- 시각화를 위한 Library

tomotopy :
- https://bab2min.github.io/tomotopy/v0.8.0/kr/#tomotopy
- 토픽 모델링을 사용하기 위한 Library


In [ ]:
!pip install tomotopy &> /dev/null
!pip install pyldavis==2.1.2 &> /dev/null #버전이 코랩과 충돌....
#!pip install pyldavis &> /dev/null #버전이 코랩과 충돌....
!pip install konlpy &> /dev/null

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git &> /dev/null
!bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh &> /dev/null

# Import Library
**tomotopy** 
* tomotopy는 토픽 모델링 툴인 tomoto의 Python 확장 버전입니다. tomoto는 c++로 작성된 깁스 샘플링 기반의 토픽 모델링 라이브러리로, 최신 CPU의 벡터화 기술을 활용하여 처리 속도를 최대로 끌어올렸습니다. 
* https://bab2min.github.io/tomotopy/v0.8.1/kr/

**pyLDAvis**
* LDAvis 는 토픽 모델링에 자주 이용되는 Latent Dirichlet Allocation (LDA) 모델의 학습 결과를 시각적으로 표현하는 라이브러리입니다.

**Konlpy (코엔엘파이)**
* 한국어에 특화된 형태소 분석 라이브러리. 여러개의 클래스가 포함됨. 
* Hannanum, Khma, Komoran, Mecab, Okt(Twitter)

**%matplotlib inline**

* %matplotlib inline 의 역할은 notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것 입니다.
출처: https://korbillgates.tistory.com/85


In [ ]:

import pandas as pd
import numpy as np

import tomotopy as tp
import pyLDAvis

from konlpy.tag import Mecab 
from konlpy.corpus import kolaw
from multiprocessing import Pool

import matplotlib.pyplot as plt
import matplotlib.colors as clr
%matplotlib inline 

# Load Dataframe

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/소셜빅데이터분석_2021/W11_LDA/donga_2019_2020.csv")

In [ ]:
df.isnull().sum()

Unnamed: 0    0
Title         0
Content       0
Date          0
dtype: int64

df=df.dropna() 도 마찬가지 기능

In [ ]:
df.dropna(inplace=True) # df = df.dropna
df.isnull().sum()

Unnamed: 0    0
Title         0
Content       0
Date          0
dtype: int64

# Tokenizer
Mecab 품사 태그 : https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit#gid=589544265

In [ ]:
mecab = Mecab()
tags = ['NNG', 'NNP']
stopwords = []

def Tokenizer(text):
    morphs = mecab.pos(text)
    pos = []

    for word, tag in morphs:
        if(len(word) < 2 and word not in stopwords): 
            continue
        if tag in tags:
            pos.append(word)

    return pos

https://bab2min.github.io/tomotopy/v0.8.1/kr/#tomotopy.LDAModel.train  
# LDAModel
**tp.LDAModel** 
- tw : int or TermWeight  
용어 가중치 기법을 나타내는 TermWeight의 열거값. 기본값은 TermWeight.ONE 입니다.
- min_cf : int  
단어의 최소 장서 빈도. 전체 문헌 내의 출현 빈도가 min_cf보다 작은 단어들은 모델에서 제외시킵니다. 기본값은 0으로, 이 경우 어떤 단어도 제외되지 않습니다.
- rm_top : int  
제거될 최상위 빈도 단어의 개수. 만약 너무 흔한 단어가 토픽 모델 상위 결과에 등장해 이를 제거하고 싶은 경우, 이 값을 1 이상의 수로 설정하십시오. 기본값은 0으로, 이 경우 최상위 빈도 단어는 전혀 제거되지 않습니다.
- k : int  
토픽의 개수, 1 ~ 32767 범위의 정수.
- alpha : float  
문헌-토픽 디리클레 분포의 하이퍼 파라미터
- eta : float  
토픽-단어 디리클레 분포의 하이퍼 파라미터
- seed : int  
난수의 시드값. 기본값은 C++의 std::random_device{}이 생성하는 임의의 정수입니다. 이 값을 고정하더라도 train시 workers를 2 이상으로 두면, 멀티 스레딩 과정에서 발생하는 우연성 때문에 실행시마다 결과가 달라질 수 있습니다.


---

**add_doc**  
현재 모델에 새로운 문헌을 추가하고 추가된 문헌의 인덱스 번호를 반환합니다.

**mdl.ll_per_word**  
현재 모델의 단어당 로그 가능도

**train**
- iter : int  
깁스 샘플링의 반복 횟수
- workers : int  
깁스 샘플링을 수행하는 데에 사용할 스레드의 개수입니다. 만약 이 값을 0으로 설정할 경우 시스템 내의 가용한 모든 코어가 사용됩니다.
- parallel : int or ParallelScheme  
학습에 사용할 병렬화 방법. 기본값은 ParallelScheme.DEFAULT로 이는 모델에 따라 최적의 방법을 tomotopy가 알아서 선택하도록 합니다.

In [ ]:
mdl = tp.LDAModel(k=10, min_cf=500, rm_top=10)
# mdl = tp.HDPModel(initial_k=10)
for text in df['Content']:  
    mdl.add_doc(Tokenizer(text))

for i in range(0, 100, 10):
    mdl.train(iter=10, workers=0)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

for k in range(mdl.k):
    print('Top 10 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=10))

Iteration: 0	Log-likelihood: -6.414894472575845
Iteration: 10	Log-likelihood: -6.140836468290769
Iteration: 20	Log-likelihood: -6.0470994913489
Iteration: 30	Log-likelihood: -6.001722145683749
Iteration: 40	Log-likelihood: -5.982702330501705
Iteration: 50	Log-likelihood: -5.970837721429501
Iteration: 60	Log-likelihood: -5.961955971650532
Iteration: 70	Log-likelihood: -5.960178565667756
Iteration: 80	Log-likelihood: -5.953620435598729
Iteration: 90	Log-likelihood: -5.955080388987517
Top 10 words of topic #0
[('코로나', 0.02391650341451168), ('사회', 0.02023201808333397), ('열병식', 0.01704579032957554), ('확장', 0.0163639634847641), ('사거리', 0.016101723536849022), ('중량', 0.016023050993680954), ('탄두', 0.015983715653419495), ('지원', 0.015078984200954437), ('여성', 0.013977572321891785), ('한국', 0.013912011869251728)]
Top 10 words of topic #1
[('대통령', 0.07721567153930664), ('국민', 0.06947734951972961), ('정부', 0.028499888256192207), ('개혁', 0.02524937316775322), ('정치', 0.02130681276321411), ('국회', 0.0206776

# Visualize
https://pyldavis.readthedocs.io/en/latest/modules/API.html  


- np.stack:  
https://numpy.org/doc/stable/reference/generated/numpy.stack.html  
- get_topic_word_dist:  
토픽 topic_id의 단어 분포를 반환합니다. 반환하는 값은 현재 토픽 내 각각의 단어들의 발생확률을 나타내는 len(vocabs)개의 소수로 구성된 list입니다.
- get_topic_dist:  
현재 문헌의 토픽 확률 분포를 list 형태로 반환합니다.

In [40]:
topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq

**pyLDAvis**  
- topic_term_dists: array-like, shape (n_topics, n_terms)  
Matrix of topic-term probabilities. Where n_terms is len(vocab).  

- doc_topic_dists :array-like, shape (n_docs, n_topics)  
Matrix of document-topic probabilities.

- doc_lengths :array-like, shape n_docs  
The length of each document, i.e. the number of words in each document. The order of the numbers should be consistent with the ordering of the docs in doc_topic_dists.

- vocab :array-like, shape n_terms  
List of all the words in the corpus used to train the model.

- term_frequency :array-like, shape n_terms  
The count of each particular term over the entire corpus. The ordering of these counts should correspond with vocab and topic_term_dists.

In [43]:
prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency,
    mds='mmds'
)


#  LDAvis 그림 해석방법

*  LDAvis 는 두 가지 정보를 출력함. 첫째 HTML 의 왼쪽에 출력되는 topic 의 2 차원 embedding vector임. 비슷한 위치에 존재하는 토픽들은 서로 비슷한 문맥을 공유함. 둘째, 오른쪽에 출력되는 각 토픽의 키워드 임. 
*  왼쪽 그림: LDAvis가 Principal Component Analysis (PCA) 를 수행한 것임 (위에서는 mmds를 수행). n_terms 차원의 벡터들을 2차원으로 압축. 모든 토픽에 자주 등장하는 단어는 정보력이 떨어지므로 중요하게 고려되지 않음. 반면, 토픽 간에 다른 패턴으로 등장하는 단어들이 중요하므로 이들을 중심으로 2차원 지도의 좌표를 학습함.
*  오른쪽 그림: HTML 의 오른쪽 상단에 λ(람다) 값은 [0, 1] 사이에서 조절가능함. 키워드 랭킹을 정하기 위해 이렇게 조절하는 것임. λ를 1 로 설정하면 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미임. λ를 0 에 가깝게 설정할수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미임. keyword score의 이같은 두가지 기준의 weight 를 조율하여 새로운 키워드를 선택할 수 있는 것임. 
*  키워드의 두가지 기준: 첫째는 salience 임. 한 토픽의 키워드라면, 각 토픽에 속한 보다 많은 문서들에서 등장해야 함 즉 P(w|t)가 커야 함. 둘째는 분별력임 (discriminative power). P(w|t)가 가장 높은 단어는 관사, 조사 등일것임(‘a, the, -은, -는, -이, -가’). 하지만 관사, 조사는 어떤 토픽을 명확히 지시해주지 못함. 차별성이 없는 단어인 것임. 따라서 LDAvis는 P(w|t)를 P(w)로 나누어 줌. 한 토픽에서 자주 등장하는 단어라 하더라도 본래 자주 등장하는 단어라면 그 중요도를 낮추겠다는 의미임. (**TF-IDF 의 관점과 비슷. 분자는 전체 문서의 개수이고, 분모는 t가 들어있는 문서의 개수입니다. t가 모든 문서에서 등장하면 IDF는 1이고, 이는 t가 별 의미 없다는 의미임).  tfidf(t,d,D)=tf(t,d)×idf(t,D)​ 
*  (다음의 출처에서 변형: https://lovit.github.io/nlp/2018/09/27/pyldavis_lda/#topic=0&lambda=1&term=). https://m.blog.naver.com/duqrlwjddns1/221780750830




In [44]:
pyLDAvis.display(prepared_data)